In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ru_synonym_dictionary = np.load('ru_syn_dict_in_wv.npy', allow_pickle=True).item()

In [ ]:
ru_synonym_dictionary['ЛУНА']

['месяц', 'полумесяц']

In [ ]:
list(ru_synonym_dictionary.items())[:3]

[('ЛУНА', ['месяц', 'полумесяц']),
 ('НЕБО', ['небосвод', 'небосклон']),
 ('ПОДНЕБЕСЬЕ', ['высота', 'вышина', 'высь'])]

In [ ]:
synonims = []
for key, value in ru_synonym_dictionary.items():
  syn_set = []
  syn_set.append(key.lower())
  syn_set.extend(value)
  synonims.append(syn_set)


In [ ]:
len(synonims)

2658

In [ ]:
synonims[:10]

[['луна', 'месяц', 'полумесяц'],
 ['небо', 'небосвод', 'небосклон'],
 ['поднебесье', 'высота', 'вышина', 'высь'],
 ['берег', 'взморье', 'побережье', 'поморье', 'прибрежье', 'брег'],
 ['болото', 'топь', 'трясина', 'болотина', 'зыбун'],
 ['долина', 'впадина', 'низина', 'низменность', 'дол'],
 ['изгиб', 'извив', 'извилина', 'изворот', 'излом', 'излучина'],
 ['континент', 'материк'],
 ['луг', 'лужок', 'поляна', 'полянка', 'луговина'],
 ['почва', 'земля', 'грунт']]

In [ ]:
ru_cbow_normalized_dict = np.load('ru_cbow_normalized_dict.npy', allow_pickle=True).item()

In [ ]:
synonims_embs = []
for syn_set in synonims:
  syn_set_embs = []
  for word in syn_set:
    emb = ru_cbow_normalized_dict[word]
    syn_set_embs.append(emb)
  synonims_embs.append(syn_set_embs)

In [ ]:
len(synonims_embs[0])

3

In [ ]:
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm
from typing import Dict, Tuple

def get_hole_diam_stats(synonym_groups: list, metric: str = 'cosine') -> Tuple[Dict[int, float], Dict[str, float]]:
    """
    Вычисляет диаметры "дыр" для каждой группы синонимов и их статистику.

    Параметры:
    - synonym_groups: список списков эмбеддингов (один подсписок = группа синонимов).
    - metric: метрика расстояния ('cosine' или 'euclidean').

    Возвращает:
    - Tuple[individual_diams, stats], где:
        - individual_diams: словарь {индекс_группы: диаметр_дыры},
        - stats: словарь {'min': мин. диаметр, 'mean': средний, 'max': макс.}.
    """
    individual_diams = {}
    all_diameters = []

    for idx, group in enumerate(tqdm(synonym_groups)):
        if len(group) < 2:
            individual_diams[idx] = 0.0  # или np.nan
            continue

        embeddings = np.array(group)
        pairwise_dist = squareform(pdist(embeddings, metric=metric))
        diameter = np.max(pairwise_dist)

        individual_diams[idx] = diameter
        all_diameters.append(diameter)

    # Вычисляем статистику (если есть хотя бы одна дыра)
    stats = {
        'min': float(np.min(all_diameters)) if all_diameters else 0.0,
        'mean': float(np.mean(all_diameters)) if all_diameters else 0.0,
        'max': float(np.max(all_diameters)) if all_diameters else 0.0
    }

    return individual_diams, stats

In [ ]:
diameters, stats = get_hole_diam_stats(synonims_embs, metric='cosine')

#print("Диаметры отдельных дыр:", diameters)
print("Статистика:", stats)

100%|██████████| 2658/2658 [00:00<00:00, 39290.00it/s]

Статистика: {'min': 0.07234445566323378, 'mean': 0.7021189322530147, 'max': 1.3032482338843099}
